In approach 2, no list variables are used. Everything is converted to categorical variables 
and later converted to entity embedding using the fastai lib
list variables like the SKU is split into parts, and the first 5 parts are converted into SKU1, SKU2, ...

In [1]:
import azureml.core
from azureml.core import Workspace, Datastore

import pandas as pd
import numpy as np

from azureml.core import Dataset

In [2]:
ws = Workspace.from_config()

All raw data stored in Azure blob container, so access thru a blob_datastore

In [3]:
blob_datastore_name='ivrdatasept' # Name of the Datastore  to workspace
container_name=os.getenv("BLOB_CONTAINER", "ivrdatasept") # Name of Azure blob container
account_name=os.getenv("BLOB_ACCOUNTNAME", "ivrdatasepton") # Storage account name
account_key=os.getenv("BLOB_ACCOUNT_KEY", "nZ12xgN95SfJTEzf+eNbnh89Ld6ZxJcMxIQ63AOVqZtwLk4ABZZkWO8BHkzT8iyZYqaIL5HQyOhkFykPg8SBAQ==") # Storage account key

blob_datastore = Datastore.register_azure_blob_container(workspace=ws, 
                                                         datastore_name=blob_datastore_name, 
                                                         container_name=container_name, 
                                                         account_name=account_name,
                                                         account_key=account_key)

In [4]:
# from azureml.core import Datastore
# from azureml.data.data_reference import DataReference
# from azureml.pipeline.core import PipelineData

# # def_blob_store = Datastore(ws, "workspaceblobstore")

# input_data = DataReference(
#        datastore=blob_datastore,
#        data_reference_name="WR_OrderProcessing_Extract_0501_1201.zip",
#        path_on_datastore="")

# output = PipelineData("output", datastore=def_blob_store)
    

In [5]:
# type(input_data)

The order processing data

In [6]:
order_extract_df = pd.read_csv('ivr_data/AWR_OrderProcessing_Extract_0501_1201_v3.zip')

/anaconda/envs/azureml_py36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (10,14,92) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [150]:
order_extract_df[order_extract_df['ACCOUNT_NUMBER']==1621175991]

,ORDER_ID,ORDER_CREATED_DATE,ACCOUNT_NUMBER,COMCAST_ACCOUNT_NUMBER,FRAUDNET_SCORE,TOTAL_PRICE,SKU,TYPE1,MAKE1,MODEL1,...,XM_ACCOUNT_OPENDATE,WRITEOFFIND,COLLECTIONSTATUS,ACCOUNT_OPEN_TO_ORDER_MINS,START_TO_CLOSE_MINS,START_TO_CASECLOSE_MINS,CASECLOSE_TO_ORDERCLOSE_MINS,CASEOPEN_TO_CASECLOSE_MINS,ORDER_RISK_LABEL,ORDER_RISK_REASON


In [153]:
order_account_list = order_extract_df['ACCOUNT_NUMBER'].unique().tolist()

In [154]:
with open('ivr_backup_data/order_account_list.txt', 'w') as filehandle:
    for account in order_account_list:
        filehandle.write('%s\n' % account)

In [155]:
order_comcast_account_list = order_extract_df['COMCAST_ACCOUNT_NUMBER'].unique().tolist()

In [156]:
with open('ivr_backup_data/order_comcast_account_list.txt', 'w') as filehandle:
    for account in order_comcast_account_list:
        filehandle.write('%s\n' % account)

In [151]:
order_extract_df[['ACCOUNT_NUMBER', 'COMCAST_ACCOUNT_NUMBER']].head()

,ACCOUNT_NUMBER,COMCAST_ACCOUNT_NUMBER
0,716308354,3364280765
1,2387138088,2735549480
2,2297289602,3659356712
3,2747556495,2019265105
4,3159523017,4140934545


In [7]:
all_order_list = order_extract_df.columns.tolist()

In [8]:
all_order_list

['ORDER_ID',
 'ORDER_CREATED_DATE',
 'ACCOUNT_NUMBER',
 'COMCAST_ACCOUNT_NUMBER',
 'FRAUDNET_SCORE',
 'TOTAL_PRICE',
 'SKU',
 'TYPE1',
 'MAKE1',
 'MODEL1',
 'PRICE1',
 'TYPE2',
 'MAKE2',
 'MODEL2',
 'PRICE2',
 'TYPE3',
 'MAKE3',
 'MODEL3',
 'PRICE3',
 'TYPE4',
 'MAKE4',
 'MODEL4',
 'PRICE4',
 'TYPE5',
 'MAKE5',
 'MODEL5',
 'PRICE5',
 'NUM_PORTIN',
 'PORT_IN_FLAG',
 'FROM_CARRIER_NAME',
 'ACNT_BILL_FNAME_MATCHES',
 'ACNT_BILL_LNAME_MATCHES',
 'ACNT_SHIP_FNAME_MATCHES',
 'ACNT_SHIP_LNAME_MATCHES',
 'SHIP_BILL_FNAME_MATCHES',
 'SHIP_BILL_LNAME_MATCHES',
 'BILL_SHIP_ADDR_MATCHES',
 'BILLING_CITY',
 'BILLING_STATE',
 'BILLING_ZIPCODE',
 'SHIPPING_CITY',
 'SHIPPING_STATE',
 'SHIPPING_ZIPCODE',
 'EMAIL',
 'IP_ADDRESS',
 'SALES_CHANNEL',
 'PHONE_MATCHES',
 'NUM_BYOD',
 'BYOD_INDICATOR',
 'FIRST_PARTY_ID_SCORE',
 'THIRD_PARTY_ID_SCORE',
 'ORDER_TYPE',
 'BLACKLIST_EMAIL',
 'FPF_HIGH_YELLOW',
 'TPF_HIGH_YELLOW',
 'SSN_BLACKLISTED',
 'DEVICE_AT_HOME',
 'ORDER_EXCHANGE',
 'YELLOW_SKU',
 'EXPERIAN_R

In [9]:
order_extract_df['ACTIVE_WATCH_RESULT'].head()     # use this one for prediction target

0      RED
1    GREEN
2    GREEN
3    GREEN
4    GREEN
Name: ACTIVE_WATCH_RESULT, dtype: object

In [128]:
order_extract_df['ACTIVE_WATCH_RESULT'].unique()

array(['RED', 'GREEN', 'YELLOW', 'BLACK'], dtype=object)

In [11]:
order_extract_df[order_extract_df['ACTIVE_WATCH_RESULT'].isna()].shape

(3684, 103)

In [12]:
order_extract_df[order_extract_df['ACCOUNT_NUMBER'].isna()].shape

(0, 103)

In [13]:
order_extract_df.shape

(247838, 103)

In [14]:
order_extract_df.dropna(subset=['ACTIVE_WATCH_RESULT', 'SHIPPING_STATE', 'BILLING_STATE'], inplace=True)

In [15]:
order_extract_df.shape

(244134, 103)

In [16]:
# n = 'ACTIVE_WATCH_RESULT'
# categories = {}
# order_extract_df.loc[:,n] = order_extract_df.loc[:,n].astype('category').cat.as_ordered()
# categories[n] = order_extract_df[n].cat.categories
# order_extract_df.loc[:,n] = pd.Categorical(order_extract_df[n], categories=categories[n], ordered=True)

Entity embedding training requires a predictive task. This can be any arbitrary task that leverages the input variables
For the order data, a convenient target variable if ACTIVE_WATCH_RESULT

In [129]:
target_df = order_extract_df[['ACTIVE_WATCH_RESULT']]

In [130]:
target_df.columns

Index(['ACTIVE_WATCH_RESULT'], dtype='object')

The fastai lib expects the label field to be numerical (for both regression and, in this case, classifcation)
Here, we convert the values Red, Green, Yellow, and Black to 0, 1, 2, and 3

In [131]:
def convert_label(label_string):
    if label_string == 'RED':
        return 3
    if label_string == 'YELLOW':
        return 2
    if label_string == 'GREEN':
        return 1
    return 3       # BLACK
    

In [132]:
target_df['ACTIVE_WATCH_RESULT'] = target_df['ACTIVE_WATCH_RESULT'].apply(lambda x: convert_label(x))

/anaconda/envs/azureml_py36/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [133]:
target_df['ACTIVE_WATCH_RESULT'].unique()

array([3, 1, 2])

In [134]:
target_df.shape

(244134, 1)

In [135]:
# save for later retrieval
target_df.to_csv('ivr_backup_data/target_df.csv', header=True, index=False)

Next we split the input variables into categorial and numerical types

In [24]:
order_extract_df.head()

,ORDER_ID,ORDER_CREATED_DATE,ACCOUNT_NUMBER,COMCAST_ACCOUNT_NUMBER,FRAUDNET_SCORE,TOTAL_PRICE,SKU,TYPE1,MAKE1,MODEL1,...,XM_ACCOUNT_OPENDATE,WRITEOFFIND,COLLECTIONSTATUS,ACCOUNT_OPEN_TO_ORDER_MINS,START_TO_CLOSE_MINS,START_TO_CASECLOSE_MINS,CASECLOSE_TO_ORDERCLOSE_MINS,CASEOPEN_TO_CASECLOSE_MINS,ORDER_RISK_LABEL,ORDER_RISK_REASON
0,187983116,2019-05-25 12:52:01,716308354,3364280765,NaN,1554.60,"[""723755019478"",""723755019478"",""190198451835"",...",Phone,Apple,iPhone 8,...,2018-12-08 10:26:59,R,NaN,241525.00,NaN,NaN,NaN,NaN,1,Did not clear Fraud Review
1,225253548,2019-10-19 19:32:01,2387138088,2735549480,NaN,0.00,"[""APLIPHONE6GOLD64""]",Phone,Apple,iPhone 6,...,2019-06-10 05:57:06,R,Not in Collection,188914.90,8988.82,NaN,NaN,NaN,0,Good Order
2,215951933,2019-09-13 15:18:21,2297289602,3659356712,NaN,1080.09,"[""190199392588""]",Phone,Apple,iPhone 11 Pro,...,2019-09-13 03:02:21,R,Not in Collection,195.95,14938.45,NaN,NaN,NaN,0,Good Order
3,217300035,2019-09-18 21:36:05,2747556495,2019265105,NaN,593.97,"[""190198453655""]",Phone,Apple,iPhone 8 Plus,...,2017-11-18 08:23:19,N,Not in Collection,963612.72,4198.95,NaN,NaN,NaN,0,Good Order
4,222110639,2019-10-07 02:25:39,3159523017,4140934545,NaN,0.00,"[""190198155795""]",Phone,Apple,iPhone 7 Plus,...,2019-09-15 11:00:12,R,Not in Collection,30625.43,15954.00,NaN,NaN,NaN,0,Good Order


In [25]:
order_extract_df.columns

Index(['ORDER_ID', 'ORDER_CREATED_DATE', 'ACCOUNT_NUMBER',
       'COMCAST_ACCOUNT_NUMBER', 'FRAUDNET_SCORE', 'TOTAL_PRICE', 'SKU',
       'TYPE1', 'MAKE1', 'MODEL1',
       ...
       'XM_ACCOUNT_OPENDATE', 'WRITEOFFIND', 'COLLECTIONSTATUS',
       'ACCOUNT_OPEN_TO_ORDER_MINS', 'START_TO_CLOSE_MINS',
       'START_TO_CASECLOSE_MINS', 'CASECLOSE_TO_ORDERCLOSE_MINS',
       'CASEOPEN_TO_CASECLOSE_MINS', 'ORDER_RISK_LABEL', 'ORDER_RISK_REASON'],
      dtype='object', length=103)

The fields that are available during order entry are marked in the Risk_Model.csv. Extracted earlier.

In [26]:
available_fields = [line.rstrip('\n') for line in open('ivr_data/available_fields.txt')]

In [27]:
available_fields.remove('ALL_SKUS')
available_fields.append('SKU')

In [28]:
available_fields

['MBC_ORDER_ID',
 'ORDER_CREATED_DATE',
 'ACCOUNT_NUMBER',
 'STATUS',
 'FRAUDNET_SCORE',
 'TOTAL_PRICE',
 'MODEL1',
 'TYPE1',
 'MAKE1',
 'PRICE1',
 'MODEL2',
 'TYPE2',
 'MAKE2',
 'PRICE2',
 'MODEL3',
 'TYPE3',
 'MAKE3',
 'PRICE3',
 'MODEL4',
 'TYPE4',
 'MAKE4',
 'PRICE4',
 'MODEL5',
 'TYPE5',
 'MAKE5',
 'PRICE5',
 'PORT_IN_FLAG',
 'NUM_PORTIN',
 'FROM_CARRIER_NAME',
 'ACNT_BILL_FNAME_MATCHES',
 'ACNT_BILL_LNAME_MATCHES',
 'ACNT_SHIP_FNAME_MATCHES',
 'ACNT_SHIP_LNAME_MATCHES',
 'SHIP_BILL_FNAME_MATCHES',
 'SHIP_BILL_LNAME_MATCHES',
 'BILL_SHIP_ADDR_MATCHES',
 'EMAIL',
 'BILLING_CITY',
 'BILLING_STATE',
 'BILLING_ZIPCODE',
 'SHIPPING_CITY',
 'SHIPPING_STATE',
 'SHIPPING_ZIPCODE',
 'SERVICE_CITY',
 'SERVICE_STATE',
 'SERVICE_ZIPCODE',
 'IP_ADDRESS',
 'SALES_CHANNEL',
 'FULFILLMENT_METHOD',
 'PHONE_MATCHES',
 'ORDER_PHONE_NUMBER',
 'NUM_BYOD',
 'BYOD_INDICATOR',
 'IS_EXISTING_CUSTOMER',
 'CART_TYPE',
 'INSTALLMENT_AMOUNT',
 'ONETIMECHARGE',
 'MONTHLYRECURRINGCHARGE',
 'SHIPPINGMETHOD',
 'L

Remove fields that Bharath did not extract and adjust column name as needed.

In [29]:
available_fields.remove('MBC_ORDER_ID')
available_fields.remove('STATUS')
# available_fields.remove('ALL_SKUS')
available_fields.remove('SERVICE_CITY')
available_fields.remove('SERVICE_STATE')
available_fields.remove('SERVICE_ZIPCODE')
available_fields.remove('FULFILLMENT_METHOD')
available_fields.remove('ORDER_PHONE_NUMBER')
available_fields.remove('CART_TYPE')
available_fields.remove('SSN_MISMATCH')
available_fields.remove('ZIP_MISMATCH')
available_fields.remove('ACCOUNT_OPENDATE')

# 'MBC_ORDER_ID' 'STATUS' 'ALL_SKUS' 'SERVICE_CITY' 'SERVICE_STATE'\n 'SERVICE_ZIPCODE' 'FULFILLMENT_METHOD' 'ORDER_PHONE_NUMBER' 'CART_TYPE'\n 'SSN_MISMATCH' 'ZIP_MISMATCH' 'ACCOUNT_OPENDATE'

In [30]:
good_order_df = order_extract_df[available_fields]

In [31]:
good_order_df.shape

(244134, 74)

In [32]:
good_order_df['ORDER_CREATED_DATE'].head()

0    2019-05-25 12:52:01
1    2019-10-19 19:32:01
2    2019-09-13 15:18:21
3    2019-09-18 21:36:05
4    2019-10-07 02:25:39
Name: ORDER_CREATED_DATE, dtype: object

In [33]:
good_order_df.columns

Index(['ORDER_CREATED_DATE', 'ACCOUNT_NUMBER', 'FRAUDNET_SCORE', 'TOTAL_PRICE',
       'MODEL1', 'TYPE1', 'MAKE1', 'PRICE1', 'MODEL2', 'TYPE2', 'MAKE2',
       'PRICE2', 'MODEL3', 'TYPE3', 'MAKE3', 'PRICE3', 'MODEL4', 'TYPE4',
       'MAKE4', 'PRICE4', 'MODEL5', 'TYPE5', 'MAKE5', 'PRICE5', 'PORT_IN_FLAG',
       'NUM_PORTIN', 'FROM_CARRIER_NAME', 'ACNT_BILL_FNAME_MATCHES',
       'ACNT_BILL_LNAME_MATCHES', 'ACNT_SHIP_FNAME_MATCHES',
       'ACNT_SHIP_LNAME_MATCHES', 'SHIP_BILL_FNAME_MATCHES',
       'SHIP_BILL_LNAME_MATCHES', 'BILL_SHIP_ADDR_MATCHES', 'EMAIL',
       'BILLING_CITY', 'BILLING_STATE', 'BILLING_ZIPCODE', 'SHIPPING_CITY',
       'SHIPPING_STATE', 'SHIPPING_ZIPCODE', 'IP_ADDRESS', 'SALES_CHANNEL',
       'PHONE_MATCHES', 'NUM_BYOD', 'BYOD_INDICATOR', 'IS_EXISTING_CUSTOMER',
       'INSTALLMENT_AMOUNT', 'ONETIMECHARGE', 'MONTHLYRECURRINGCHARGE',
       'SHIPPINGMETHOD', 'LINE_LIMIT_REACHED', 'DENIAL_OF_SERVICES',
       'BYOD_RETRY_COUNT', 'PREORDER_LINES', 'ALLOWED_LINES',


In [34]:
# good_order_df[good_order_df['ACCOUNT_NUMBER']==2173973859]

There are 40,934 zipcodes in US. Drop it for now

In [35]:
good_order_df.drop(['BILLING_ZIPCODE'], axis=1, inplace=True)
good_order_df.drop(['SHIPPING_ZIPCODE'], axis=1, inplace=True)

/anaconda/envs/azureml_py36/lib/python3.6/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


Also drop the cities. Too sparse to incorporate

In [36]:
good_order_df.drop(['SHIPPING_CITY'], axis=1, inplace=True)
good_order_df.drop(['BILLING_CITY'], axis=1, inplace=True)

Check the colunmns with nan. There all need to be handled

In [37]:
NaN_list = good_order_df.columns[good_order_df.isna().any()].tolist()

In [38]:
len(NaN_list)

27

In [39]:
NaN_list

['FRAUDNET_SCORE',
 'PRICE1',
 'MODEL2',
 'TYPE2',
 'MAKE2',
 'PRICE2',
 'MODEL3',
 'TYPE3',
 'MAKE3',
 'PRICE3',
 'MODEL4',
 'TYPE4',
 'MAKE4',
 'PRICE4',
 'MODEL5',
 'TYPE5',
 'MAKE5',
 'PRICE5',
 'PORT_IN_FLAG',
 'FROM_CARRIER_NAME',
 'PREORDER_LINES',
 'INTERNAL_CREDIT_CHECK_DONE',
 'ORDER_EXCHANGE',
 'EXPERIAN_RESPONSE_1',
 'EXPERIAN_RESPONSE_2',
 'IDA_RESULT',
 'FRAUDNET_RESULT']

In [40]:
# good_order_df.dtypes

Handle numerical and categorical variables separately. Numerical fields first

In [41]:
order_num_df = good_order_df.select_dtypes(include=np.number)

In [42]:
num_columns = order_num_df.columns.tolist()

There were string values captured in PRICE1 and PRICE2, so they didn't get picked up automatically. have to manually incorported them

In [43]:
num_columns.append('PRICE1')
num_columns.append('PRICE2')
# ORDER_CREATED_DATE is a numerical field after conversion to unix time
num_columns.append('ORDER_CREATED_DATE')

num_columns.remove('ACCOUNT_NUMBER')     # ACCOUNT_NUMBER should not be treated as a number


In [44]:
num_columns

['FRAUDNET_SCORE',
 'TOTAL_PRICE',
 'PRICE3',
 'PRICE4',
 'PRICE5',
 'NUM_PORTIN',
 'ACNT_BILL_FNAME_MATCHES',
 'ACNT_BILL_LNAME_MATCHES',
 'ACNT_SHIP_FNAME_MATCHES',
 'ACNT_SHIP_LNAME_MATCHES',
 'SHIP_BILL_FNAME_MATCHES',
 'SHIP_BILL_LNAME_MATCHES',
 'BILL_SHIP_ADDR_MATCHES',
 'EMAIL',
 'IP_ADDRESS',
 'PHONE_MATCHES',
 'NUM_BYOD',
 'INSTALLMENT_AMOUNT',
 'ONETIMECHARGE',
 'MONTHLYRECURRINGCHARGE',
 'BYOD_RETRY_COUNT',
 'PREORDER_LINES',
 'ALLOWED_LINES',
 'APPROVED_LINES',
 'EXTERNAL_APPROVED_LINES',
 'FIRST_PARTY_ID_SCORE',
 'THIRD_PARTY_ID_SCORE',
 'PRICE1',
 'PRICE2',
 'ORDER_CREATED_DATE']

Create a separate DataFrame to process the numerical columns

In [45]:
order_num_df = good_order_df[num_columns]

Which of the numerical columns contain nan?

In [46]:
order_num_df.columns[order_num_df.isna().any()].tolist()

['FRAUDNET_SCORE',
 'PRICE3',
 'PRICE4',
 'PRICE5',
 'PREORDER_LINES',
 'PRICE1',
 'PRICE2']

Fill nan values with 0

In [47]:
order_num_df['PREORDER_LINES'].fillna(0, inplace=True)
order_num_df['FRAUDNET_SCORE'].fillna(0, inplace=True)
# order_num_df['PRICE1'].fillna(0, inplace=True)
# order_num_df['PRICE2'].fillna(0, inplace=True)
order_num_df['PRICE3'].fillna(0, inplace=True)
order_num_df['PRICE4'].fillna(0, inplace=True)
order_num_df['PRICE5'].fillna(0, inplace=True)

/anaconda/envs/azureml_py36/lib/python3.6/site-packages/pandas/core/generic.py:5434: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


Double check 0's got filled in

In [48]:
order_num_df[['FRAUDNET_SCORE', 'PRICE3','PRICE4','PRICE5', 'PREORDER_LINES']].head()

,FRAUDNET_SCORE,PRICE3,PRICE4,PRICE5,PREORDER_LINES
0,0.0,119.99,119.99,0.0,0.0
1,0.0,0.00,0.00,0.0,0.0
2,0.0,0.00,0.00,0.0,0.0
3,0.0,0.00,0.00,0.0,0.0
4,0.0,0.00,0.00,0.0,0.0


Convert PRICE1 and PRICES2 to numerical values

In [49]:
def convert_to_float(price_string):
    try:
        return float(price_string)
    except:
        print('price_string={}'.format(price_string))
        return 0

In [50]:
order_num_df[['PRICE1', 'PRICE2']].head()

,PRICE1,PRICE2
0,599.99,599.99
1,0.00,NaN
2,999.99,NaN
3,549.99,NaN
4,469.99,NaN


In [51]:
order_num_df['PRICE1'] = order_num_df['PRICE1'].apply(lambda x: convert_to_float(x))

price_string=153 NGP
price_string=153 NGP
price_string=153 NGP
price_string=153 NGP
price_string=153 NGP
price_string=153 NGP
price_string=153 NGP
price_string=153 NGP
price_string=153 NGP
price_string=153 NGP
price_string=153 NGP
price_string=153 NGP
price_string=153 NGP
price_string=153 NGP
price_string=153 NGP
price_string=153 NGP
price_string=153 NGP
price_string=153 NGP
price_string=153 NGP
price_string=153 NGP
price_string=153 NGP


/anaconda/envs/azureml_py36/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [52]:
order_num_df['PRICE2'] = order_num_df['PRICE2'].apply(lambda x: convert_to_float(x))

price_string=153 NGP


/anaconda/envs/azureml_py36/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [53]:
# order_num_df['PRICE1'].isna()
order_num_df['PRICE1'].fillna(0, inplace=True)
order_num_df['PRICE2'].fillna(0, inplace=True)

PRICE1 and PRICE2 should now be all numerical, and have 0's where they used to be nan

In [54]:
order_num_df[['PRICE1', 'PRICE2']].head()

,PRICE1,PRICE2
0,599.99,599.99
1,0.00,0.00
2,999.99,0.00
3,549.99,0.00
4,469.99,0.00


In [55]:
# order_num_df.iloc[100]['ORDER_CREATED_DATE']

In [56]:
import time
import datetime

def convert_to_unix_time(date_time_string):
    return time.mktime(datetime.datetime.strptime(date_time_string, "%Y-%m-%d %H:%M:%S").timetuple())


In [57]:
order_num_df['ORDER_CREATED_DATE'] = order_num_df['ORDER_CREATED_DATE'].apply(lambda x: convert_to_unix_time(x))

# order_num_df['ORDER_CREATED_DATE_vector'].head()

# order_num_df.drop(['ORDER_CREATED_DATE'], axis=1, inplace=True)

/anaconda/envs/azureml_py36/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [148]:
order_num_df['ORDER_CREATED_DATE'].head()

0    1.558789e+09
1    1.571514e+09
2    1.568388e+09
3    1.568843e+09
4    1.570415e+09
Name: ORDER_CREATED_DATE, dtype: float64

In [58]:
order_num_df.columns[order_num_df.isna().any()].tolist()

[]

In [147]:
order_num_df.shape

(244134, 30)

In [60]:
order_num_df.to_csv('ivr_backup_data/order_num_df.csv', index=False)

Get all the numerical column names

In [61]:
num_columns

['FRAUDNET_SCORE',
 'TOTAL_PRICE',
 'PRICE3',
 'PRICE4',
 'PRICE5',
 'NUM_PORTIN',
 'ACNT_BILL_FNAME_MATCHES',
 'ACNT_BILL_LNAME_MATCHES',
 'ACNT_SHIP_FNAME_MATCHES',
 'ACNT_SHIP_LNAME_MATCHES',
 'SHIP_BILL_FNAME_MATCHES',
 'SHIP_BILL_LNAME_MATCHES',
 'BILL_SHIP_ADDR_MATCHES',
 'EMAIL',
 'IP_ADDRESS',
 'PHONE_MATCHES',
 'NUM_BYOD',
 'INSTALLMENT_AMOUNT',
 'ONETIMECHARGE',
 'MONTHLYRECURRINGCHARGE',
 'BYOD_RETRY_COUNT',
 'PREORDER_LINES',
 'ALLOWED_LINES',
 'APPROVED_LINES',
 'EXTERNAL_APPROVED_LINES',
 'FIRST_PARTY_ID_SCORE',
 'THIRD_PARTY_ID_SCORE',
 'PRICE1',
 'PRICE2',
 'ORDER_CREATED_DATE']

In [62]:
len(num_columns)

30

One way to get all the categorical columns is use the statement below

In [63]:
good_order_df.select_dtypes(['object', 'category']).columns

Index(['ORDER_CREATED_DATE', 'MODEL1', 'TYPE1', 'MAKE1', 'PRICE1', 'MODEL2',
       'TYPE2', 'MAKE2', 'PRICE2', 'MODEL3', 'TYPE3', 'MAKE3', 'MODEL4',
       'TYPE4', 'MAKE4', 'MODEL5', 'TYPE5', 'MAKE5', 'PORT_IN_FLAG',
       'FROM_CARRIER_NAME', 'BILLING_STATE', 'SHIPPING_STATE', 'SALES_CHANNEL',
       'BYOD_INDICATOR', 'IS_EXISTING_CUSTOMER', 'SHIPPINGMETHOD',
       'LINE_LIMIT_REACHED', 'DENIAL_OF_SERVICES',
       'INTERNAL_CREDIT_CHECK_DONE', 'EXTERNAL_CREDIT_CHECK_DONE',
       'BLACKLIST_EMAIL', 'FPF_HIGH_YELLOW', 'TPF_HIGH_YELLOW',
       'SSN_BLACKLISTED', 'DEVICE_AT_HOME', 'ORDER_EXCHANGE', 'YELLOW_SKU',
       'EXPERIAN_RESPONSE_1', 'EXPERIAN_RESPONSE_2', 'IDA_RESULT',
       'FRAUDNET_RESULT', 'SKU'],
      dtype='object')

But let me instead get all columns and subtract the numerical ones. That way I am sure no columns got skipped

In [64]:
all_columns = good_order_df.columns.tolist()

In [65]:
all_columns

['ORDER_CREATED_DATE',
 'ACCOUNT_NUMBER',
 'FRAUDNET_SCORE',
 'TOTAL_PRICE',
 'MODEL1',
 'TYPE1',
 'MAKE1',
 'PRICE1',
 'MODEL2',
 'TYPE2',
 'MAKE2',
 'PRICE2',
 'MODEL3',
 'TYPE3',
 'MAKE3',
 'PRICE3',
 'MODEL4',
 'TYPE4',
 'MAKE4',
 'PRICE4',
 'MODEL5',
 'TYPE5',
 'MAKE5',
 'PRICE5',
 'PORT_IN_FLAG',
 'NUM_PORTIN',
 'FROM_CARRIER_NAME',
 'ACNT_BILL_FNAME_MATCHES',
 'ACNT_BILL_LNAME_MATCHES',
 'ACNT_SHIP_FNAME_MATCHES',
 'ACNT_SHIP_LNAME_MATCHES',
 'SHIP_BILL_FNAME_MATCHES',
 'SHIP_BILL_LNAME_MATCHES',
 'BILL_SHIP_ADDR_MATCHES',
 'EMAIL',
 'BILLING_STATE',
 'SHIPPING_STATE',
 'IP_ADDRESS',
 'SALES_CHANNEL',
 'PHONE_MATCHES',
 'NUM_BYOD',
 'BYOD_INDICATOR',
 'IS_EXISTING_CUSTOMER',
 'INSTALLMENT_AMOUNT',
 'ONETIMECHARGE',
 'MONTHLYRECURRINGCHARGE',
 'SHIPPINGMETHOD',
 'LINE_LIMIT_REACHED',
 'DENIAL_OF_SERVICES',
 'BYOD_RETRY_COUNT',
 'PREORDER_LINES',
 'ALLOWED_LINES',
 'INTERNAL_CREDIT_CHECK_DONE',
 'EXTERNAL_CREDIT_CHECK_DONE',
 'APPROVED_LINES',
 'EXTERNAL_APPROVED_LINES',
 'FIRST_P

In [66]:
# one field in order_num_columns is not in all_columns
# for num_column in num_columns:
#     if num_column not in all_columns:
#         print('{} not in all_columns'.format(num_column))

In [67]:
len(all_columns)

70

In [68]:
object_columns = list(set(all_columns) - set(num_columns))

In [69]:
object_columns

['MODEL1',
 'MAKE2',
 'BILLING_STATE',
 'TYPE3',
 'TYPE2',
 'BLACKLIST_EMAIL',
 'YELLOW_SKU',
 'MODEL4',
 'TPF_HIGH_YELLOW',
 'ACCOUNT_NUMBER',
 'EXTERNAL_CREDIT_CHECK_DONE',
 'IS_EXISTING_CUSTOMER',
 'FRAUDNET_RESULT',
 'MAKE4',
 'LINE_LIMIT_REACHED',
 'MAKE1',
 'IDA_RESULT',
 'EXPERIAN_RESPONSE_1',
 'EXPERIAN_RESPONSE_2',
 'MODEL2',
 'TYPE1',
 'SKU',
 'SHIPPING_STATE',
 'BYOD_INDICATOR',
 'SALES_CHANNEL',
 'TYPE4',
 'MAKE5',
 'MODEL5',
 'DEVICE_AT_HOME',
 'SSN_BLACKLISTED',
 'PORT_IN_FLAG',
 'MAKE3',
 'TYPE5',
 'DENIAL_OF_SERVICES',
 'FPF_HIGH_YELLOW',
 'SHIPPINGMETHOD',
 'INTERNAL_CREDIT_CHECK_DONE',
 'ORDER_EXCHANGE',
 'MODEL3',
 'FROM_CARRIER_NAME']

In [70]:
len(object_columns)

40

Next, create a DataFrame for all the categorical columns only

In [71]:
order_category_df = good_order_df[object_columns]

In [72]:
order_category_df.shape

(244134, 40)

In [73]:
# delete the orginal df to save memory
del good_order_df

In [74]:
# order_category_df['IS_EXISTING_CUSTOMER'].unique()

Start of list variable conversion - The SKU and BYOD variables must be broken up into 5 separate fields.  

The SKU field is one of the complicated ones, because it is a list of devices. Each devices can come from a long list of available products. For the purpose of entity embedding computation, this list is broken up into 5 separate SKUn fields (only take the first 5 if the list is longer)

In [75]:
sku_list = order_category_df['SKU'].unique().tolist()

In [76]:
sku_list

['["723755019478","723755019478","190198451835","190198451781"]',
 '["APLIPHONE6GOLD64"]',
 '["190199392588"]',
 '["190198453655"]',
 '["190198155795"]',
 '["190199220577","190199380677"]',
 '["190198066497","190198066534"]',
 '["190198456700"]',
 '["652810830621"]',
 '["190199392564"]',
 '["190198456656"]',
 '["APLIPH8PSIL64N1X"]',
 '["APLIPH6PLUSSIL16"]',
 '["190199392526"]',
 '["887276363332"]',
 '["190198354426"]',
 '["887276348490"]',
 '["190199380714"]',
 '["190198786029","191058084972"]',
 '["190198776440","848467076929"]',
 '["190199220652"]',
 '["887276246512"]',
 '["APLIPHONE6SILV16"]',
 '["888462734868"]',
 '["190199220553"]',
 '["190199220539"]',
 '["190199220652","190199220737"]',
 '["190199220751"]',
 '["190198776426"]',
 '["190199380738"]',
 '["APLIPHONE6GRAY64","SEASMG950UBLCK64"]',
 '["190198451835"]',
 '["190198043375"]',
 '["APLIPHONE6GRAY32"]',
 '["190199392663"]',
 '["190198058614"]',
 '["887276348490","887276348490"]',
 '["APLIPH8GOLD64N1X"]',
 '["888462501958","A

Take an inventory of all devices. Hopefully, this is full list. Otherwise, trouble down the road.

In [77]:

def collect_all_skus(sku_string):
#     print('sku_string={}'.format(sku_string))
    skus = [np.nan] * 9
#     sku_2 = np.nan
#     sku_3 = np.nan
#     sku_4 = np.nan
#     sku_5 = np.nan
    truncated_string = sku_string[1:-1]
#     print('truncated_string={}'.format(truncated_string))
    sku_array = truncated_string.split(',')
    no_skus = len(sku_array)
    i = 0
    for sku in sku_array:
#         print('skus len={}'.format(len(skus)))
#         print('i = {}'.format(i))
        sku = sku.replace("\"", "")
        try:
#             print('sku={}'.format(sku))
#             skus[i] = sku
            all_skus.add(sku)
        except Exception as e:
            print('sku_string={}'.format(sku_string))
            print("I/O error({})".format(e))
        i += 1
#         if i > 3:
#             break
        
#     return pd.Series([skus[0], skus[1], skus[2], skus[3], skus[4]],index=['SKU_1','SKU_2','SKU_3','SKU_4','SKU_5'])


In [78]:
all_skus = set()

for row in order_category_df.iterrows():
#     print(row[1])
#     print(row[1]['SKU'])
    collect_all_skus(row[1]['SKU'])

In [79]:
len(all_skus)

609

In [80]:
def split_skus(sku_string):
#     print('sku_string={}'.format(sku_string))
    skus = [np.nan] * 9

    truncated_string = sku_string[1:-1]
#     print('truncated_string={}'.format(truncated_string))
    sku_array = truncated_string.split(',')
    no_skus = len(sku_array)
    i = 0
    for sku in sku_array:
#         print('skus len={}'.format(len(skus)))
#         print('i = {}'.format(i))
        sku = sku.replace("\"", "")
        try:
#             print('sku={}'.format(sku))
            skus[i] = sku
        except Exception as e:
            print('sku_string={}'.format(sku_string))
            print("I/O error({})".format(e))
        i += 1
#         if i > 3:
#             break
#     return [skus[0], skus[1], skus[2], skus[3], skus[4]]
    return pd.Series([skus[0], skus[1], skus[2], skus[3], skus[4]],index=['SKU1','SKU2','SKU3','SKU4','SKU5'])


In [81]:
# test_category_df = order_category_df.copy()

In [82]:
# test_category_df = test_category_df.sample(n=100)

In [83]:
sku_df = order_category_df['SKU'].apply(lambda x: split_skus(x))

In [84]:
sku_df.head()

,SKU1,SKU2,SKU3,SKU4,SKU5
0,723755019478,723755019478,190198451835,190198451781,NaN
1,APLIPHONE6GOLD64,NaN,NaN,NaN,NaN
2,190199392588,NaN,NaN,NaN,NaN
3,190198453655,NaN,NaN,NaN,NaN
4,190198155795,NaN,NaN,NaN,NaN


In [85]:
sku_df.shape

(244134, 5)

In [86]:
order_category_df = pd.concat(
    [
        order_category_df,
        sku_df
    ], axis=1
)

In [87]:
order_category_df.head()

,MODEL1,MAKE2,BILLING_STATE,TYPE3,TYPE2,BLACKLIST_EMAIL,YELLOW_SKU,MODEL4,TPF_HIGH_YELLOW,ACCOUNT_NUMBER,...,SHIPPINGMETHOD,INTERNAL_CREDIT_CHECK_DONE,ORDER_EXCHANGE,MODEL3,FROM_CARRIER_NAME,SKU1,SKU2,SKU3,SKU4,SKU5
0,iPhone 8,Apple,MN,Phone,Phone,N,N,moto e5 play,N,716308354,...,2Days,Y,NaN,moto e5 play,"[null,null,null,null]",723755019478,723755019478,190198451835,190198451781,NaN
1,iPhone 6,NaN,MN,NaN,NaN,N,N,NaN,N,2387138088,...,2Days,Y,NaN,NaN,"[""MetroPCS""]",APLIPHONE6GOLD64,NaN,NaN,NaN,NaN
2,iPhone 11 Pro,NaN,MN,NaN,NaN,N,N,NaN,N,2297289602,...,2Days,Y,NaN,NaN,"[""GoogleVoice""]",190199392588,NaN,NaN,NaN,NaN
3,iPhone 8 Plus,NaN,MN,NaN,NaN,N,N,NaN,N,2747556495,...,2Days,NaN,NaN,NaN,[null],190198453655,NaN,NaN,NaN,NaN
4,iPhone 7 Plus,NaN,MN,NaN,NaN,N,N,NaN,N,3159523017,...,2Days,Y,NaN,NaN,"[""Other/NotSure""]",190198155795,NaN,NaN,NaN,NaN


In [88]:
order_category_df.shape

(244134, 45)

Next also convert BYOD_INDICATOR to the same 5 element categorical variable format

In [89]:
order_category_df['BYOD_INDICATOR'].head()

0    [" "," "," "," "]
1                ["Y"]
2               [null]
3               [null]
4                ["Y"]
Name: BYOD_INDICATOR, dtype: object

In [90]:
# def generate_device_vector(row):
def split_byods(sku_string, byod_string):
#     print('row = {}'.format(row))
#     print('sku_string={}'.format(sku_string))
#     print('byod_string={}'.format(byod_string))
#     SKU_vector = [0] * (device_count + 1)
    byod = [np.nan] * 5

    truncated_string = sku_string[1:-1]
# #     print('truncated_string={}'.format(truncated_string))
    sku_array = truncated_string.split(',')
    truncated_byod_string = byod_string[1:-1]
    byod_array = truncated_byod_string.split(',')
#     no_skus = len(sku_array)
    i = 0
    for byod_string in byod_array:
# #         print('skus len={}'.format(len(skus)))
# #         print('i = {}'.format(i))
        byod_string = byod_string.replace("\"", "").replace("''", "")
        sku = sku_array[i].replace("\"", "")
        try:
#             print('sku={}'.format(sku))
#             print('byod={}'.format(byod))         
            if byod_string.lower() == 'y':
#                 print('found byod')
                byod[i] = sku     # if byod, add 10
        except Exception as e:
            if i < 5:
                print('sku_string={}'.format(sku_string))
                print('byod_string={}'.format(byod_string))
                print("I/O error({})".format(e))
        i += 1
#         if i > 3:
#             break
        
    return pd.Series([byod[0], byod[1], byod[2], byod[3], byod[4]],index=['BYOD1','BYOD2','BYOD3','BYOD4','BYOD5'])


In [91]:
byod_df = order_category_df.apply(lambda x: split_byods(x['SKU'], x['BYOD_INDICATOR']), axis=1)

In [92]:
byod_df.head()

,BYOD1,BYOD2,BYOD3,BYOD4,BYOD5
0,NaN,NaN,NaN,NaN,NaN
1,APLIPHONE6GOLD64,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN
4,190198155795,NaN,NaN,NaN,NaN


In [93]:
byod_df.shape

(244134, 5)

In [94]:
order_category_df = pd.concat(
    [
        order_category_df,
        byod_df
    ], axis=1
)

In [96]:
# order_category_df.shape

We don't need the original columns SKU and BYOD_INDICATOR any more. Their values are now represented by SKU_vector

In [97]:
order_category_df.drop(['SKU', 'BYOD_INDICATOR'], axis=1, inplace=True)

In [98]:
order_category_df.columns

Index(['MODEL1', 'MAKE2', 'BILLING_STATE', 'TYPE3', 'TYPE2', 'BLACKLIST_EMAIL',
       'YELLOW_SKU', 'MODEL4', 'TPF_HIGH_YELLOW', 'ACCOUNT_NUMBER',
       'EXTERNAL_CREDIT_CHECK_DONE', 'IS_EXISTING_CUSTOMER', 'FRAUDNET_RESULT',
       'MAKE4', 'LINE_LIMIT_REACHED', 'MAKE1', 'IDA_RESULT',
       'EXPERIAN_RESPONSE_1', 'EXPERIAN_RESPONSE_2', 'MODEL2', 'TYPE1',
       'SHIPPING_STATE', 'SALES_CHANNEL', 'TYPE4', 'MAKE5', 'MODEL5',
       'DEVICE_AT_HOME', 'SSN_BLACKLISTED', 'PORT_IN_FLAG', 'MAKE3', 'TYPE5',
       'DENIAL_OF_SERVICES', 'FPF_HIGH_YELLOW', 'SHIPPINGMETHOD',
       'INTERNAL_CREDIT_CHECK_DONE', 'ORDER_EXCHANGE', 'MODEL3',
       'FROM_CARRIER_NAME', 'SKU1', 'SKU2', 'SKU3', 'SKU4', 'SKU5', 'BYOD1',
       'BYOD2', 'BYOD3', 'BYOD4', 'BYOD5'],
      dtype='object')

In [99]:
order_category_df.head()

,MODEL1,MAKE2,BILLING_STATE,TYPE3,TYPE2,BLACKLIST_EMAIL,YELLOW_SKU,MODEL4,TPF_HIGH_YELLOW,ACCOUNT_NUMBER,...,SKU1,SKU2,SKU3,SKU4,SKU5,BYOD1,BYOD2,BYOD3,BYOD4,BYOD5
0,iPhone 8,Apple,MN,Phone,Phone,N,N,moto e5 play,N,716308354,...,723755019478,723755019478,190198451835,190198451781,NaN,NaN,NaN,NaN,NaN,NaN
1,iPhone 6,NaN,MN,NaN,NaN,N,N,NaN,N,2387138088,...,APLIPHONE6GOLD64,NaN,NaN,NaN,NaN,APLIPHONE6GOLD64,NaN,NaN,NaN,NaN
2,iPhone 11 Pro,NaN,MN,NaN,NaN,N,N,NaN,N,2297289602,...,190199392588,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,iPhone 8 Plus,NaN,MN,NaN,NaN,N,N,NaN,N,2747556495,...,190198453655,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,iPhone 7 Plus,NaN,MN,NaN,NaN,N,N,NaN,N,3159523017,...,190198155795,NaN,NaN,NaN,NaN,190198155795,NaN,NaN,NaN,NaN


In [100]:
# order_category_df.shape

PORT_IN_FLAG and FROM_CARRIER_NAME are two other fields that should be combined

In [101]:
order_category_df[['PORT_IN_FLAG', 'FROM_CARRIER_NAME']].head()

,PORT_IN_FLAG,FROM_CARRIER_NAME
0,"[""N"",""N"",""N"",""N""]","[null,null,null,null]"
1,"[""Y""]","[""MetroPCS""]"
2,"[""Y""]","[""GoogleVoice""]"
3,"[""N""]",[null]
4,"[""Y""]","[""Other/NotSure""]"


Looks like FROM_CARRIER_NAME only contains valid data when PORT_IN_FLAG is Y, so no need to make use of PORT_IN_FLAG

In [102]:
def isNaN(num):
    return num != num

In [103]:
def generate_carrier_vector(carrier_string):
#     print('row = {}'.format(row))

    carrier_vector = [np.nan] * 5       # 5 carriers max. 0 means null

    try:
        truncated_string = carrier_string[1:-1]
    except:
#         print('carrier_string={}'.format(carrier_string))
        return pd.Series([carrier_vector[0], carrier_vector[1], carrier_vector[2], carrier_vector[3], carrier_vector[4]],index=['CARRIER1','CARRIER2','CARRIER3','CARRIER4','CARRIER5'])

# #     print('truncated_string={}'.format(truncated_string))
    carrier_array = truncated_string.split(',')
    i = 0
    for carrier in carrier_array:
        carrier = carrier.replace("\"", "")

        try:
#             print('carrier={}'.format(carrier))
            if carrier == "null":
                pass       # leave as np.nan
#                 print('got null')
#             print('byod={}'.format(byod))
            else:
                carrier_vector[i] = carrier
        except Exception as e:
            print('carrier={}'.format(carrier))
            print("I/O error({})".format(e))
        i += 1
        
    return pd.Series([carrier_vector[0], carrier_vector[1], carrier_vector[2], carrier_vector[3], carrier_vector[4]],index=['CARRIER1','CARRIER2','CARRIER3','CARRIER4','CARRIER5'])


In [104]:
carrier_df = order_category_df.apply(lambda x: generate_carrier_vector(x['FROM_CARRIER_NAME']), axis=1)

In [105]:
carrier_df.head()

,CARRIER1,CARRIER2,CARRIER3,CARRIER4,CARRIER5
0,NaN,NaN,NaN,NaN,NaN
1,MetroPCS,NaN,NaN,NaN,NaN
2,GoogleVoice,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN
4,Other/NotSure,NaN,NaN,NaN,NaN


In [109]:
# carrier_df.shape

In [107]:
order_category_df = pd.concat(
    [
        order_category_df,
        carrier_df
    ], axis=1
)

In [110]:
# order_category_df.shape

In [111]:
order_category_df.drop(['FROM_CARRIER_NAME','PORT_IN_FLAG'], axis=1, inplace=True)

leave Make, Type and Model all as single value category variables to be tranformed using fasti databunch

how many of the categorical fields have Nan left? They will be handled by fastai Categorify function

In [112]:
NaN_cat_list = order_category_df.columns[order_category_df.isna().any()].tolist()

In [113]:
NaN_cat_list

['MAKE2',
 'TYPE3',
 'TYPE2',
 'MODEL4',
 'FRAUDNET_RESULT',
 'MAKE4',
 'IDA_RESULT',
 'EXPERIAN_RESPONSE_1',
 'EXPERIAN_RESPONSE_2',
 'MODEL2',
 'TYPE4',
 'MAKE5',
 'MODEL5',
 'MAKE3',
 'TYPE5',
 'INTERNAL_CREDIT_CHECK_DONE',
 'ORDER_EXCHANGE',
 'MODEL3',
 'SKU2',
 'SKU3',
 'SKU4',
 'SKU5',
 'BYOD1',
 'BYOD2',
 'BYOD3',
 'BYOD4',
 'BYOD5',
 'CARRIER1',
 'CARRIER2',
 'CARRIER3',
 'CARRIER4',
 'CARRIER5']

Calling out the state fields as a potential missing category problem

In [120]:
state_list = order_category_df['SHIPPING_STATE'].unique()

In [115]:
state_list

array(['MN', 'MA', 'WI', 'CA', 'NH', 'ME', 'CT', 'VT', 'TX', 'NY', 'OR',
       'WA', 'PA', 'WV', 'MD', 'OH', 'GA', 'FL', 'DE', 'VA', 'DC', 'SC',
       'LA', 'TN', 'MS', 'AR', 'AL', 'KY', 'IN', 'UT', 'NC', 'AZ', 'CO',
       'IL', 'NM', 'NJ', 'MI', 'MO', 'KS', 'ID'], dtype=object)

In [116]:
len(state_list)

40

In [121]:
billing_state_list = order_category_df['BILLING_STATE'].unique()

In [122]:
billing_state_list

array(['MN', 'UT', 'MA', 'PA', 'WI', 'CA', 'NY', 'CT', 'KY', 'NH', 'FL',
       'NC', 'TX', 'MI', 'VA', 'ME', 'DC', 'VT', 'NJ', 'LA', 'CO', 'MD',
       'OR', 'WA', 'NV', 'NM', 'ID', 'AZ', 'OK', 'IL', 'IN', 'GA', 'WV',
       'IA', 'OH', 'AL', 'KS', 'TN', 'DE', 'SC', 'MS', 'AR', 'SD', 'NE',
       'HI', 'MO', 'MT', 'RI', 'AK', 'WY', 'ND'], dtype=object)

In [123]:
len(billing_state_list)    # this one is ok

51

In [117]:
order_category_df[order_category_df['SHIPPING_STATE'].isna()].shape

(0, 51)

In [118]:
order_category_df[order_category_df['BILLING_STATE'].isna()].shape

(0, 51)

In [112]:
# only 41 states show up in data. This is potentially a problem using fastai lib which only use categorical values present 
# in the data
# if a new state show up in the future, may bomb out -> requires new training

In [113]:
# order_category_df.dropna(subset=['SHIPPING_STATE'], inplace=True)

In [115]:
state_list = [line.rstrip('\n') for line in open('ivr_data/states.txt')]

In [116]:
# the full state list should have 51 possible values
len(state_list)

51

all remaining single categorical fields to be fed to fastai model as is. fastai transform functions will categorize them

In [124]:
order_category_df.columns

Index(['MODEL1', 'MAKE2', 'BILLING_STATE', 'TYPE3', 'TYPE2', 'BLACKLIST_EMAIL',
       'YELLOW_SKU', 'MODEL4', 'TPF_HIGH_YELLOW', 'ACCOUNT_NUMBER',
       'EXTERNAL_CREDIT_CHECK_DONE', 'IS_EXISTING_CUSTOMER', 'FRAUDNET_RESULT',
       'MAKE4', 'LINE_LIMIT_REACHED', 'MAKE1', 'IDA_RESULT',
       'EXPERIAN_RESPONSE_1', 'EXPERIAN_RESPONSE_2', 'MODEL2', 'TYPE1',
       'SHIPPING_STATE', 'SALES_CHANNEL', 'TYPE4', 'MAKE5', 'MODEL5',
       'DEVICE_AT_HOME', 'SSN_BLACKLISTED', 'MAKE3', 'TYPE5',
       'DENIAL_OF_SERVICES', 'FPF_HIGH_YELLOW', 'SHIPPINGMETHOD',
       'INTERNAL_CREDIT_CHECK_DONE', 'ORDER_EXCHANGE', 'MODEL3', 'SKU1',
       'SKU2', 'SKU3', 'SKU4', 'SKU5', 'BYOD1', 'BYOD2', 'BYOD3', 'BYOD4',
       'BYOD5', 'CARRIER1', 'CARRIER2', 'CARRIER3', 'CARRIER4', 'CARRIER5'],
      dtype='object')

In [125]:
order_category_df.shape

(244134, 51)

In [126]:
order_category_df.to_csv('ivr_backup_data/order_category_df.csv', index=False)

play with groupby ACCOUNT_NUMBER to see how many accounts we are dealing with
This is potentially one of the most interesting variable to cluster, once entity embedding is derived

In [136]:
len(order_extract_df['ACCOUNT_NUMBER'].unique())

195857

In [141]:
order_by_account = order_extract_df.groupby('ACCOUNT_NUMBER')

In [142]:
order_by_account.size().unique()

array([ 1,  4,  2,  3,  9,  6,  5,  7,  8, 10, 12, 11, 13, 16, 14, 20, 18,
       19, 46, 22, 28, 15, 23])

In [143]:
# i = 1
# for group_name, df_group in order_by_account:
#     print('group name={}'.format(group_name))
#     print('group size={}'.format(df_group.size))
#     i += 1
#     if i > 1:
#         break

In [144]:
group_count_df = order_by_account.size().reset_index(name='size')

In [146]:
group_count_df[group_count_df['size']==46]

,ACCOUNT_NUMBER,size
99451,2173973859,46


In [138]:
order_extract_df[order_extract_df['ACCOUNT_NUMBER']==2173973859]

,ORDER_ID,ORDER_CREATED_DATE,ACCOUNT_NUMBER,COMCAST_ACCOUNT_NUMBER,FRAUDNET_SCORE,TOTAL_PRICE,SKU,TYPE1,MAKE1,MODEL1,...,XM_ACCOUNT_OPENDATE,WRITEOFFIND,COLLECTIONSTATUS,ACCOUNT_OPEN_TO_ORDER_MINS,START_TO_CLOSE_MINS,START_TO_CASECLOSE_MINS,CASECLOSE_TO_ORDERCLOSE_MINS,CASEOPEN_TO_CASECLOSE_MINS,ORDER_RISK_LABEL,ORDER_RISK_REASON
236228,221807253,2019-10-05 19:30:32,2173973859,1030251582,NaN,1430.76,"[""190199392687""]",Phone,Apple,iPhone 11 Pro,...,2019-08-10 13:13:06,R,Suspended due to Collection,80477.42,NaN,4039.97,NaN,3965.95,1,Did not clear Fraud Review
236229,218569896,2019-09-23 17:06:02,2173973859,1030251582,NaN,1536.84,"[""190199380875""]",Phone,Apple,iPhone 11 Pro Max,...,2019-08-10 13:13:06,R,Suspended due to Collection,63052.90,NaN,4285.33,NaN,4249.30,1,Did not clear Fraud Review
236231,216824896,2019-09-16 21:55:54,2173973859,1030251582,4000.0,1536.84,"[""190199380875""]",Phone,Apple,iPhone 11 Pro Max,...,2019-08-10 13:13:06,R,Suspended due to Collection,53262.78,1055.87,2495.48,1439.62,2385.82,1,Did not clear Fraud Review
236232,217432630,2019-09-19 13:40:06,2173973859,1030251582,NaN,1536.84,"[""190199380875""]",Phone,Apple,iPhone 11 Pro Max,...,2019-08-10 13:13:06,R,Suspended due to Collection,57086.98,NaN,4849.08,NaN,4807.50,1,Did not clear Fraud Review
236234,221805343,2019-10-05 19:19:13,2173973859,1030251582,NaN,1430.76,"[""190199392687""]",Phone,Apple,iPhone 11 Pro,...,2019-08-10 13:13:06,R,Suspended due to Collection,80466.08,NaN,4048.53,NaN,4002.50,1,Did not clear Fraud Review
236235,207673186,2019-08-12 16:53:49,2173973859,1030251582,2100.0,2649.84,"[""190198786180"",""190198786180""]",Phone,Apple,iPhone XS Max,...,2019-08-10 13:13:06,R,Suspended due to Collection,2560.67,NaN,2871.27,NaN,2842.98,1,Did not clear Fraud Review
236236,220152739,2019-09-29 18:55:13,2173973859,1030251582,NaN,168.49,"[""190199098428""]",Accessory,Apple,Accessory,...,2019-08-10 13:13:06,R,Suspended due to Collection,71802.12,NaN,NaN,NaN,NaN,1,Did not clear Fraud Review
236237,216996131,2019-09-17 17:46:35,2173973859,1030251582,NaN,1536.84,"[""190199380875""]",Phone,Apple,iPhone 11 Pro Max,...,2019-08-10 13:13:06,R,Suspended due to Collection,54453.47,NaN,3340.00,NaN,3180.70,1,Did not clear Fraud Review
236238,220315644,2019-09-30 14:02:39,2173973859,1030251582,NaN,1536.84,"[""190199380875""]",Phone,Apple,iPhone 11 Pro Max,...,2019-08-10 13:13:06,R,Suspended due to Collection,72949.53,1966.38,6513.67,4547.28,6480.02,1,Did not clear Fraud Review
236239,212844591,2019-09-01 20:17:11,2173973859,1030251582,2500.0,2331.84,"[""190198786067"",""190198786067""]",Phone,Apple,iPhone XS Max,...,2019-08-10 13:13:06,R,Suspended due to Collection,31564.07,NaN,NaN,NaN,NaN,1,Did not clear Fraud Review


In [228]:
min(order_extract_df['ORDER_CREATED_DATE'])

'2019-04-30 00:00:22'

In [229]:
max(order_extract_df['ORDER_CREATED_DATE'])

'2019-11-30 23:59:04'

In [230]:
order_by_account.ngroups

198233

In [231]:
order_category_df.columns

Index(['SKU_vector', 'carrier_vector', 'make_vector', 'type_vector',
       'model_vector', 'SHIPPING_STATE_cat', 'BILLING_STATE_cat',
       'LINE_LIMIT_REACHED_cat', 'SSN_BLACKLISTED_cat',
       'EXPERIAN_RESPONSE_1_cat', 'EXPERIAN_RESPONSE_2_cat', 'IDA_RESULT_cat',
       'INTERNAL_CREDIT_CHECK_DONE_cat', 'TPF_HIGH_YELLOW_cat',
       'EXTERNAL_CREDIT_CHECK_DONE_cat', 'IS_EXISTING_CUSTOMER_cat',
       'SALES_CHANNEL_cat', 'SHIPPINGMETHOD_cat', 'FPF_HIGH_YELLOW_cat',
       'FRAUDNET_RESULT_cat', 'YELLOW_SKU_cat', 'DEVICE_AT_HOME_cat',
       'ORDER_EXCHANGE_cat', 'DENIAL_OF_SERVICES_cat'],
      dtype='object')